### TensorFlow: "Integrating Machine Learning Systems into User-Facing Software"### 

**By [Patrick Hebron](http://www.patrickhebron.com)**

In [ ]:
# Import a few system libraries:
import os, os.path, json, time

# Import Numpy library:
import numpy as np

# Import TensorFlow library:
import tensorflow as tf

# Import Flask library:
from flask import Flask, render_template, jsonify, request, url_for, send_from_directory
from werkzeug import secure_filename

In [ ]:
# Set path to imagenet data directory:
model_dir = 'data/imagenet_retrain'

In [ ]:
# Image Recognizer:
class ImageRecognizer(object):
    # Initialize image recognition model:
    def __init__(self):
        self.idx_to_human = self.loadLookupTable(
            os.path.join(model_dir, 'output_labels.txt'))
        self.graph = self.createGraph(
            os.path.join(model_dir, 'output_graph.pb'))
        self.session = tf.Session(graph=self.graph)

    # Define human-readable string lookup function:
    def loadLookupTable(self,file_path):
        if not tf.gfile.Exists(file_path):
            tf.logging.fatal('File does not exist %s', file_path)
        idx_to_human = []
        file_lines = tf.gfile.GFile(file_path).readlines()
        for line in file_lines:
            idx_to_human.append( line.rstrip('\n') )
        return idx_to_human

    # Define graph loader function:
    def createGraph(self,file_path):
        with tf.gfile.FastGFile(file_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            graph = tf.import_graph_def(graph_def, name='')
            return graph

    # Define image prediction function:
    def predict(self,image_path,num_top_predictions=5):
        if not tf.gfile.Exists( image_path ):
            tf.logging.fatal( 'File does not exist %s', image_path )
        image_data = tf.gfile.FastGFile( image_path, 'rb' ).read()
        softmax_tensor = self.session.graph.get_tensor_by_name('final_result:0')
        predictions = self.session.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
        predictions = np.squeeze(predictions)
        top_k = predictions.argsort()[-num_top_predictions:][::-1]
        output = []
        for node_id in top_k:
            name  = self.idx_to_human[node_id]
            score = predictions[node_id]
            output.append( { 'name' : name, 'score' : str( score ) } )
        return output

In [ ]:
# Initialize the Flask application:
app = Flask('app')

# Configure upload settings:
app.config['ALLOWED_EXTENSIONS'] = set(['jpg', 'jpeg'])

# Checks whether file is allowed:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in app.config['ALLOWED_EXTENSIONS']

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory('/tmp', filename)

@app.route('/upload', methods=['POST'])
def upload():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file( file.filename ):
            filename = secure_filename( file.filename )
            filepath = os.path.join( '/tmp', filename )
            fileurl  = url_for( 'uploaded_file', filename=filename )
            file.save( filepath )
            res_data = recognizer.predict( filepath )
            os.remove( filepath )
            return jsonify( { 'prediction' : res_data } )

@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
recognizer = ImageRecognizer()
app.run(host='0.0.0.0', port=8080, debug=False)